<a href="https://colab.research.google.com/github/Mgiri1234/LabProject_VLM/blob/main/VLM_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -q -U bitsandbytes
%pip install transformers datasets torchvision torch
%pip install -q pytorch_lightning  # For training


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which 

In [ ]:
pip install bitsandbytes accelerate transformers

In [ ]:
# Install dependencies for quantized model usage
# %pip install bitsandbytes accelerate transformers

from transformers import AutoProcessor, PaliGemmaForConditionalGeneration, BitsAndBytesConfig
from PIL import Image
import requests
import torch

# Model details
model_id = "google/paligemma-3b-mix-224"

# Load image from URL
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/car.jpg?download=true"
image = Image.open(requests.get(url, stream=True).raw)

# Configuration for 8-bit quantization (use 4-bit for even lower memory usage if desired)
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with quantization config for CPU
model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id,
    quantization_config=quantization_config
).eval()

# Load processor for preparing inputs
processor = AutoProcessor.from_pretrained(model_id)

# Create prompt (in Spanish for this example)
prompt = "what objects are there in the image"
model_inputs = processor(text=prompt, images=image, return_tensors="pt")

# Generate caption
input_len = model_inputs["input_ids"].shape[-1]

with torch.no_grad():
    generation = model.generate(**model_inputs, max_new_tokens=100, do_sample=False)
    generation = generation[0][input_len:]
    decoded = processor.decode(generation, skip_special_tokens=True)

print("Generated caption:", decoded)


config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/62.6k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Generated caption: Car, Land vehicle, Vehicle, Wheel.


In [ ]:
pip install datasets transformers bitsandbytes accelerate pillow


In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration, BitsAndBytesConfig
from PIL import Image

# Load OpenImagesV7 dataset (segmentation task)
dataset = load_dataset("dalle-mini/open-images", split="validation")

# Model configuration
model_id = "google/paligemma-3b-mix-224"
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# Load PaliGemma model with 8-bit precision
model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id, quantization_config=quantization_config
).eval()

# Load the processor
processor = AutoProcessor.from_pretrained(model_id)

# Check if CUDA is available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Function to load and process image from dataset
def process_image(sample):
    # Load the image
    image_url = sample['image']  # OpenImagesV7 provides URLs for images
    image = Image.open(requests.get(image_url, stream=True).raw)

    # Define the segmentation task prompt
    prompt = "segment"

    # Process the image and prompt
    model_inputs = processor(text=prompt, images=image, return_tensors="pt").to(device)

    # Get the input length for text generation
    input_len = model_inputs["input_ids"].shape[-1]

    # Generate the output
    with torch.no_grad():
        generation = model.generate(**model_inputs, max_new_tokens=100, do_sample=False)
        generation = generation[0][input_len:]

    # Decode the result (segmentation output)
    decoded_output = processor.decode(generation, skip_special_tokens=True)
    return decoded_output

# Example usage: process the first image in the dataset
sample_image = dataset[0]
segmentation_result = process_image(sample_image)
print(segmentation_result)


Resolving data files:   0%|          | 0/3606 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

00014.parquet:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

00000.parquet:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/41620 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/125436 [00:00<?, ? examples/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

ValueError: `.to` is not supported for `4-bit` or `8-bit` bitsandbytes models. Please use the model as it is, since the model has already been set to the correct devices and casted to the correct `dtype`.